SPDX-FileCopyrightText: © 2025 Data Science Center, Nara Institute of Science and Technology \<naist_eln@ml.naist.ac.jp\> </br>
SPDX-License-Identifier: AGPL-3.0

# Tutorial #1: APIを用いたeLabFTWのノート作成

<b>Note:</b> ご不明な点がございましたら、lee.jemyung@naist.ac.jp にご連絡ください

## **Step 4:** APIを使ってオフラインで作成したデータを入れてみましょう（2/2）
#### XLSX形式で作成した実験データを読み込み、XLSXファイル内に指定されたテンプレートを検索、データ保存位置を探索した後、マッチングされるテーブルとカラムにデータを入力するサンプルコード

#### Note: _Step 4_ で<span style='color: orange'>新しくお知らせする部分のタイトル</span>はオレンジ色で表示し、その他の部分では _Step 1_, _2_, 又は _3_ と同じ内容となります。

## 前準備
###　「.env」ファイルの作成
「.env」ファイルの作成に関しては[**<ins>こちら</ins>**](https://github.com/jemyung/enl/blob/main/API/eLabFTW_API_env.ipynb)をご覧ください。<br>
「.env」ファイルがすでに用意されている場合は、そのまま次に進んでください。

## <span style='color: orange'>実験資料の取得</span>

### <span style='color: orange'>入力ファイルの設定</span>
実験ノートのデータが保存されたXLSXファイルを指定し、ファイル内に保存されたテンプレート情報を獲得する項目名を指定<br>
（[例：XLSX入力ファイルサンプルはこちら](https://github.com/naist-eln/eln/blob/main/API/test_file/exp_sample.xlsx)）<br>
* 「file_path」: データが保存されたXLSXファイルのフォルダーパス<br>
* 「exp_file」: データが保存されたXLSXファイル名<br>
* 「exp_items」: XLSXファイルに保存された実験ノートに関する情報（テンプレートタイトル、サンプルID 等）の項目名、各項目は：<br>
    + 「template_id」: テンプレートのID (省略可)<br>
    + 「template_title」: テンプレートのタイトル名<br>
    + 「sample_id」: サンプルID<br>
    + 「upload_file」: アップロードするファイルのパス(省略可)<br>

In [1]:
file_path = './test_file/'
exp_file = 'exp_sample.xlsx'
exp_items = ['template_id', 'template_title', 'sample_id', 'upload_file']

exp_itm_dic = dict(zip(exp_items, ['' for i in range(len(exp_items))]))

### コード駆動準備
ライブラリ、パッケージ、モジュールの呼び出しと設定

In [2]:
import os
import re
import elabapi_python
from elabapi_python.rest import ApiException
import json
from requests.exceptions import HTTPError
import numpy as np
import pandas as pd
from pandas import json_normalize
from openpyxl import load_workbook as lw
from IPython.display import display, HTML
from bs4 import BeautifulSoup
import requests
import codecs
import smbclient
from smb.SMBConnection import SMBConnection
from dotenv import load_dotenv
import warnings
from urllib3.exceptions import InsecureRequestWarning
from difflib import get_close_matches

# APIクライアントの設定
load_dotenv()
configuration = elabapi_python.Configuration()
configuration.host = os.environ["HOST_URL"]
configuration.api_key['Authorization'] = os.environ["API_KEY"]
configuration.debug = False
configuration.verify_ssl = False

# SSL警告を無効化
warnings.simplefilter('ignore', InsecureRequestWarning)

# 各種のインスタンスを作成
api_client = elabapi_python.ApiClient(configuration)
api_client.set_default_header(header_name = 'Authorization', header_value = configuration.api_key['Authorization'] )
experimentsApi = elabapi_python.ExperimentsApi(api_client)
templatesApi = elabapi_python.ExperimentsTemplatesApi(api_client)
uploadsApi = elabapi_python.UploadsApi(api_client)
itemsApi = elabapi_python.ItemsApi(api_client)

### <span style='color: orange'>使用する関数の定義</span>
テーブル名称とテンプレート内部の文字を処理するための関数を実装

In [3]:
# Remove special characters from the string 'str'
# Can define more removing character(s) by adding it at 'char'.
def rmsc(str, char = ''):
    try:
        if str:
            rv_str = re.sub(r'[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', '', str)
            if char: rv_str = rv_str.lstrip(char)
            return rv_str.strip()
        else:
            return str
    except Exception as e:
        print(f"An error occurred: {e}\n")

### <span style='color: orange'>XLSXファイルデータ取得</span>
XLSXファイルのデータを取得し、Pandasのデータフレーム（dataframe）形式で保存

In [4]:
df = pd.read_excel(file_path + exp_file, engine = 'openpyxl')

# 以下のコマンドを有効にすると、呼び込んだXLSXファイルの内容が確認可能
# display(df)

### <span style='color: orange'>取得したデータの分類</span>
データフレームで保存したデータからテンプレート情報、テーブル情報を分類し、コード内部の変数に項目別に保存

In [5]:
wb = lw(file_path + exp_file, data_only=True)
ws = wb.active

exp_contents = dict()
table_name = ''
col_ttls = []
chk_tbl, chk_ttl, chk_cnt = False, False, False

try:
    for row in ws.rows:
        for cell in row:
            if cell.value and isinstance(cell.value, str) and cell.value.startswith('#'):
                cell.value = ''
    
    for row in ws.rows:
        n_cell = len(row)
        r_val = ''
        if row[0].value:
            chk_sgl = not any([row[i].value for i in range(1, n_cell)])
            if isinstance(row[0].value, str): r_val = rmsc(row[0].value)
            if r_val in exp_items:
                if row[1].value:
                    exp_itm_dic[r_val] = str(row[1].value).strip()
            elif chk_sgl and row[0].border.right.style is None:
                table_name = r_val
                exp_contents[table_name] = []
                chk_tbl = True
            elif not chk_sgl or row[0].border.right.style is not None:
                if chk_tbl:
                    if chk_ttl:
                        col_vals = [str(cell.value) for cell in row]
                        # col_vals = [cell.value for cell in row]
                        
                        while col_vals and col_vals[-1] is None: del col_vals[-1] 
                        exp_contents[table_name].append(dict(zip(col_ttls, col_vals)))
                        chk_cnt = True
                    else:
                        col_ttls = [cell.value for cell in row]
                        while col_ttls and col_ttls[-1] is None: del col_ttls[-1] 
                        if col_ttls: chk_ttl = True
                else:
                    raise ValueError("Table title is not defined for the contents: ", [cell.value for cell in row])
        elif not any([row[i].value for i in range(n_cell)]):
            if chk_tbl:
                if chk_ttl and not chk_cnt:
                    raise ValueError(f"Table content is not defined for the table: {table_name}")
                elif not chk_ttl:
                    raise ValueError(f"Table column titles are not defined for the table: {table_name}")
                
            table_name = ''
            col_ttls = []
            chk_tbl, chk_ttl, chk_cnt = False, False, False
            
except Exception as e:
    print(f"An error occurred: {e}\n")        

### <span style='color: orange'>テンプレート情報の抽出</span>
分類されたデータからテンプレート関連情報を別途保存

In [6]:
template_id = exp_itm_dic['template_id']
template_title = exp_itm_dic['template_title']
sample_id = exp_itm_dic['sample_id']
upload_file = file_path + exp_itm_dic['upload_file']

## 実験ノート作成

### テンプレートリストの確認
現在登録されているテンプレートのリストを表示します。テンプレートのIDとTitleを確認することができます。

In [7]:
# テンプレートIDとタイトルを取得
templates_data = []
errors = []

try:
    templates_list = templatesApi.read_experiments_templates()
    
    for template in templates_list:
        try:
            templates_data.append({
                'ID': template.id,
                'Title': template.title
            })
        except AttributeError as e:
            errors.append(f"AttributeError for template ID {template.id}: {e}")
        except Exception as e:
            errors.append(f"An error occurred for template ID {template.id}: {e}")

except ApiException as e:
    print(f"Exception when calling TemplatesApi->read_experiments_templates: {e}\n")
    if e.body:
        print(f"Error details: {e.body}\n")

# データフレームに変換して表示
templates_df = pd.DataFrame(templates_data)
templates_df[['ID']] = templates_df[['ID']].astype(str)
display(templates_df)

# エラーの内容を表示
if errors:
    print("Errors encountered:")
    for error in errors:
        print(error)

,ID,Title
0,109,Test_No.1
1,108,Test_No.2


### テンプレートの割り当て
指定したIDまたはTitleからテンプレートを割り当て、IDとTitleを確定</br>
コード実行時に割り当てられたIDとTitleが表示される。</br>
<span style='color: red'>**<ins>NOTE</ins>**</span></br>
入力されたタイトルと一致するテンプレートがない場合、最も近いタイトルのテンプレートとして自動的に指定する。 </br>
案内メッセージが表示されますので、ご確認後、異常がない場合はお進みください。

In [8]:
detected_title = ""
try:
    if template_id and (templates_df['ID'] == template_id).any():
        detected_title = templates_df.loc[templates_df['ID'] == template_id, 'Title'].values[0]
        if not template_title:
            template_title = detected_title
    elif template_title:
        if (templates_df['Title'] == template_title).any():
            detected_title = template_title
        else:
            detected_title = get_close_matches(template_title, templates_df['Title'] , 3, 0.7)
            if detected_title:
                detected_title = detected_title[0]
            else:
                raise ValueError(f"Template title is not detected.")
        template_id = templates_df.loc[templates_df['Title'] == detected_title, 'ID'].values[0]
    else:
        raise ValueError("Please enter a valid template ID or Title, above.")
    
    print(f"Template title is assigned to \"{detected_title}\"" + ("." if detected_title == template_title else f", altered from \"{template_title}\"."))

    if template_id in templates_df['ID'].values:
        print(f"Template ID: {template_id}")
    else:
        raise ValueError(f"Template ID {template_id} not found in templates_df")
except ValueError as e:
    print("ValueError: %s\n" % e)

Template title is assigned to "Test_No.2", altered from "Test_No2".
Template ID: 108


### テンプレートの取得
割り当てられたIDのテンプレートを取得

In [9]:
# Prepare the body for creating an experiment
template = None
experiment_body = {
    "category_id": template_id,  # カテゴリIDを指定
}

try:
    if template_id in templates_df['ID'].values:
        template = templatesApi.get_experiment_template(template_id)
        print(f"Template (ID: {template_id}) is ready to use.")
    else:
        raise ValueError(f"Template ID {template_id} not found in templates dataframe.")
except ValueError as e:
    print("ValueError: %s\n" % e)

Template (ID: 108) is ready to use.


### テンプレートの確認
取得したテンプレートを確認

In [10]:
try:    
    template_dict = template.to_dict()
    template_html = "<h2>Template Information</h2>"
    template_html += "<ul>"
    for key, value in template_dict.items():
        template_html += f"<li><strong>{key}:</strong> {value}</li>"
    template_html += "</ul>"
    # display(HTML(template_html))
except ApiException as e:
    print(f"Exception when calling TemplatesApi->get_experiment_template: {e}\n")
    if e.body:
           print(f"Error details: {e.body}\n")
except Exception as e:
    print(f"An error occurred: {e}\n")

### 実験ノートの準備
テンプレートから「(Template title)_(Sample ID)」をタイトルにする実験ノートを作成してeLabFTWに登録する。</br>
正常に登録が完了したら、その結果を出力する。

In [11]:
try:
    # テンプレートIDがデータフレームに存在するか確認
    if template_id not in templates_df['ID'].values:
        display(templates_df)
        print(template_id, " ", type(template_id))
        raise ValueError(f"Template ID {template_id} not found in templates_df")

    # テンプレートIDに対応するタイトルを取得
    template_title = templates_df.loc[templates_df['ID'] == template_id, 'Title'].values[0]
    
    # 新しいタイトルを作成
    new_title = f"{template_title}_{sample_id}"
    
    # 実験ノートの内容を設定
    experiment_body = {
        "category_id": str(template_id)  # カテゴリIDを指定
    }
   
    # Create an experiment
    api_response = experimentsApi.post_experiment(body=experiment_body)    
    print("Experiment created successfully: ")
    if api_response is None: print(f"{api_response} issue.")
    else: print(api_response)

    # 新しい実験ノートのIDを取得するため、最新の実験ノートを取得
    experiments_list = sorted(experimentsApi.read_experiments(), key=lambda x: x._date, reverse=True)
    new_experiment_id = experiments_list[0].id

    # 実験ノートのタイトルを更新
    update_body = {
        "title": new_title
    }
    update_response = experimentsApi.patch_experiment(new_experiment_id, body=update_body)
    print("Experiment title updated successfully:")
    # print(update_response)

except ValueError as e:
    print("ValueError: %s\n" % e)
except ApiException as e:
    print("Exception when calling ExperimentsApi->post_experiment or patch_experiment: %s\n" % e)
    if e.body:
        print(f"Error details: {e.body}\n")
except Exception as e:
    print("An error occurred: %s\n" % e)

Experiment created successfully: 
None issue.
Experiment title updated successfully:


### 実験ノートリストの確認
作成した全実験ノートを出力する。

In [12]:
# 全実験ノートの確認
try:
    # デフォルト設定で実験ノートを取得
    print("デフォルト設定で実験ノートを取得:")
    experimentsList = experimentsApi.read_experiments()
    print(f"Number of experiments: {len(experimentsList)}")
    
    # 必要なフィールドを抽出してリストに格納
    exp_data = []
    for experiment in experimentsList:
        exp_dict = experiment.to_dict()
        exp_data.append({
            '_date': exp_dict.get('_date'),
            'category_title': exp_dict.get('category_title'),
            'category': exp_dict.get('category'),
            'fullname': exp_dict.get('fullname'),
            'id': exp_dict.get('id'),
            'tags': exp_dict.get('tags'),
            'tags_id': exp_dict.get('tags_id'),
            'up_item_id': exp_dict.get('up_item_id')
        })

    # データフレームに変換
    experiments_df = pd.DataFrame(exp_data)
    experiments_df['_date'] = pd.to_datetime(experiments_df['_date'])   # _date列をdatetime型に変換
    experiments_df = experiments_df.sort_values(by='_date', ascending=False)    # 作成日が新しい順にソート
    # print(experiments_df)
    
except ApiException as e:
    print("ExperimentsApiを呼び出す際の例外: %s\n" % e)

デフォルト設定で実験ノートを取得:
Number of experiments: 24


### 最新の実験ノートを取得
実験ノートリストから最新の実験ノートを取得する。</br>
取得した実験ノートのIDが表示されます。

In [13]:
try:
    experiments_list = experimentsApi.read_experiments()
    # print(f"Number of experiments: {len(experiments_list)}")
    
    # 最も新しい実験ノートを取得
    if experiments_list:
        experiments_list = sorted(experiments_list, key=lambda x: x._date, reverse=True)  # 作成日でソート
        experiment_id = experiments_list[0].id  # 最新の実験ノートのIDを取得
        print(f"Latest experiment ID: {experiment_id}")        
    else:
        print("No experiments found.")
        exit()

except ApiException as e:
    print(f"Exception when calling ExperimentsApi->read_experiments: {e}\n")
    exit()

Latest experiment ID: 2071


## <span style='color: orange'>実験ノートの内容更新</span>

### ノート更新の準備
テーブルの基本スタイルを設定し、テーブルに列を追加する関数とデータから特殊文字を書き出し、テキストだけを抽出する関数を定義

In [14]:
# Define the template's default line color and border style.
default_style = {
    'border-color' : '#95a5a6',
    'border-style' : 'solid'
}

# Add 'n_row' empty row(s) in the transferred 'table' of 'soup', 
# reflecting the row('tr_style') and cell('td_style') styles.
def addEmptyRow(table, soup, n_row = 1, tr_style = '', td_style = ''):
    
    n_cell = len(table.tr.find_all('td'))
    
    for i in range(n_row):
        empt_tr = soup.new_tag('tr', style = tr_style) if tr_style else soup.new_tag('tr')
    
        for j in range(n_cell):
            nw_td = soup.new_tag('td', style = td_style)
            nw_td.string = ' '
            empt_tr.append(nw_td)
            
        table.append(empt_tr)
    
    return table

# Remove special characters from the string 'str'
# Can define more removing character(s) by adding it at 'char'.
def removeSpChar(str, char):
    try:
        rv_str = re.sub(r'[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', '', str)
        if char: rv_str = rv_str.lstrip(char).strip()
        return rv_str
    except Exception as e:
        print(f"An error occurred: {e}\n")

### 実験ノートの内容を更新
最新の実験ノートの詳細を取得して実験ノートの内容を更新する。<br>
'exp_contents' のデータをノートの各テーブルに追加します。<br>
次のようなプロセスでテーブルにデータを追加します。<br>
   1. 'exp_contents' dictionaryの'テーブル名'に該当するテーブルをノートで検索して、<br>
   2. 名前が一致するか似たようなテーブルに各データを追加します。<br>
   3. テーブルにデータを追加するとき、「exp_contents」内に定義された「タイトル」に該当するテーブルのColumnに「入力値」を追加します。<br>
    参考：「exp_contents」は「テーブル名」: [（「タイトル」:「入力値」）] のDictionaryになっています

In [15]:
# 最新の実験ノートの詳細を取得
try:
    latest_experiment= experimentsApi.get_experiment(experiment_id)
    exp_body = latest_experiment.body
    exp_title = latest_experiment.title

    # BeautifulSoupを使ってHTMLを解析
    soup = BeautifulSoup(exp_body, 'html.parser')
    
    cnt_soup = soup.find_all(['h1','p'])
    cnt_dict = dict()
    for cs in cnt_soup:
        cst = removeSpChar(cs.text, 'TS')
        if cst:
            cnt_dict[cst] = cs

    for es in list(exp_contents.keys()):
        ec_list = exp_contents[es]
        n_exp = len(ec_list)
        
        # cnt_soup = soup.find(['h1','p'], text = re.compile('(.*)'+es+'(.*)'))   # legacy code
        cnt_soup = cnt_dict[es]        
        table = cnt_soup.find_next_sibling('table')
        
        if table:
            rows = table.find_all('tr')
            
            if len(rows) > 1:
                items = list(map(lambda x:x.text, rows[1].find_all('td')))     # Column titles (English ver.)
                n_items = len(items)

                chk_empt_row = [all([c.get_text(strip=True) in ['', None] for c in r.find_all('td')]) for r in rows]
                n_empt_row = np.count_nonzero(chk_empt_row)
                n_filled_row = len(rows) - n_empt_row
                
                # Add empty row(s) if it's insufficient
                if n_empt_row < n_exp:
                    tr_style = rows[2]['style'] if rows[2].attrs.get('style') else ''
                    td_style = rows[2].td['style']
                    
                    addEmptyRow(table, soup, n_exp - n_empt_row, tr_style, td_style)
                    rows = table.find_all('tr')
                
                # Reflect the contents
                for i, ec in enumerate(ec_list):               
                    cells = rows[i + n_filled_row].find_all('td')      # Experiment values         
                    for ec_title in ec.keys():
                        if ec_title in items:
                            cells[items.index(ec_title)].string = ec[ec_title]
                        else:
                            candidate = get_close_matches(ec_title, items, 3, 0.8)
                            candidate = candidate[0] if candidate else "None"
                            raise ValueError(f"Column {ec_title} not found in table: candidate = {candidate}")
            else:
                raise ValueError(f'Insufficient rows: size {len(rows)}')

    # 更新されたHTMLを文字列に戻す
    rev_exp_body = str(soup)

    # 新しい内容をもとに既存の内容を更新
    update_body = {
        "body": rev_exp_body,  # 更新されたHTML
        # "title": exp_title + " - 更新しました"  # タイトルを更新
    }

    # 実験ノートの内容を更新
    update_response = experimentsApi.patch_experiment(experiment_id, body=update_body)
    print("Experiment updated successfully:")
    # print(update_response)
except ApiException as e:
    print(f"Exception when calling ExperimentsApi->patch_experiment: {e}\n")
    if e.body:
        print(f"Error details: {e.body}\n")
except Exception as e:
    print(f"An error occurred: {e}\n")

Experiment updated successfully:


### <span style='color: orange'>ファイルのアップロード</span>
「upload_file」にあるファイルをノートにアップロードします。

In [16]:
try:
    if upload_file:
        uploadsApi.post_upload('experiments', experiment_id, file=upload_file, comment='Uploaded with APIv2')
        print(f'File \"{upload_file}\" has uploaded to experiment #{experiment_id}.')
    else:
        print("No file to upload")

except ApiException as e:
    print("UploadsApiを呼び出す際の例外: %s\n" % e)

File "./test_file/xlsx_upload_test.txt" has uploaded to experiment #2071.


お疲れ様でした。<br>
 [<ins>次のステップでは</ins>、]()保存された実験ノートをダウンロードするサンプル コードを見てみましょう。

Copyright (C) 2025 Data Science Center, Nara Institute of Science and Technology

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU Affero General Public License as
published by the Free Software Foundation, either version 3 of the
License, or (at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
GNU Affero General Public License for more details.

You should have received a copy of the GNU Affero General Public License
along with this program. If not, see <http://www.gnu.org/licenses/>.